In [1]:
import pandas as pd
from fuzzywuzzy import process

# Load datasets
df1 = pd.read_csv("/Users/vvmohith/Desktop/PROJECT/dataset/05-06/05-06.csv", delimiter=";", usecols=[0], encoding="utf-8")
df2 = pd.read_csv("/Users/vvmohith/Desktop/PROJECT/dataset/20-21/2020-2021.csv", delimiter=",", skiprows=7, usecols=[0], encoding="utf-8")

# Rename columns
df1.columns = ["Ministry"]
df2.columns = ["Ministry"]

# Convert to lowercase and strip spaces
df1["Ministry"] = df1["Ministry"].str.lower().str.strip()
df2["Ministry"] = df2["Ministry"].str.lower().str.strip()

# Remove specific words from 05-06 dataset
df1["Ministry"] = df1["Ministry"].str.replace(r"\b(revenue|capital)\b", "", regex=True).str.strip()

# Remove specific rows from 2020-2021 dataset
exclude_phrases = [
    "central sector schemes/projects",
    "establishment expenditure of the centre",
    "other central sector expenditure"
]
df2 = df2[~df2["Ministry"].isin(exclude_phrases)]

# Drop NaN values
df1.dropna(inplace=True)
df2.dropna(inplace=True)

# Create lists of ministries
ministries_05_06 = df1["Ministry"].unique().tolist()
ministries_20_21 = df2["Ministry"].unique().tolist()

# Fuzzy match all ministries
matches = []
for ministry in ministries_05_06:
    best_match, score = process.extractOne(ministry, ministries_20_21)
    matches.append((ministry, best_match, score))

# Convert to DataFrame
matched_df = pd.DataFrame(matches, columns=["Ministry_05_06", "Matched_Ministry_20_21", "Match_Score"])

# Sort by match score
matched_df = matched_df.sort_values(by="Match_Score", ascending=False)

# Save output to a CSV file
matched_df.to_csv("/Users/vvmohith/Desktop/PROJECT/matched_ministries.csv", index=False)

# Display all matches
print(matched_df)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


                                       Ministry_05_06  \
94                                        grand total   
43                                    law and justice   
50          personnel, public grievances and pensions   
32  ayurveda, yoga & naturopathy, unani, siddha an...   
23                development of north eastern region   
..                                                ...   
78                                        state plans   
27          economic affairs (centralised provisions)   
81                                              doner   
88              union territories without legislature   
1                                                       

                               Matched_Ministry_20_21  Match_Score  
94                                        grand total          100  
43                                64. law and justice           95  
50  73. ministry of personnel, public grievances a...           95  
32  4. ministry of ayurveda, yoga and n

In [10]:
import pandas as pd
import os

# Path to datasets
budget_data_folder = "/Users/vvmohith/Desktop/PROJECT/dataset/"  # Update with actual path
match_file = "/Users/vvmohith/Desktop/PROJECT/matched_ministries.csv"  # Matched ministries file

# Load matched ministries
df_match = pd.read_csv(match_file)
common_ministries = df_match[["Ministry_05_06", "Matched_Ministry_20_21"]]

# Initialize final dataframe
final_df = pd.DataFrame()

# Define the range of years
years_available = list(range(2005, 2022))

# Loop through available years and process the data
for year in years_available:
    folder_name = f"{str(year)[-2:]}-{str(year+1)[-2:]}"  # Folder names like "05-06"
    file_path = os.path.join(budget_data_folder, f"{folder_name}.csv")
    
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path, delimiter=",")
        except pd.errors.ParserError:
            try:
                df = pd.read_csv(file_path, delimiter=";")
            except pd.errors.ParserError:
                print(f"Skipping file due to parsing error: {file_path}")
                continue
        
        df.columns = [col.lower().strip() for col in df.columns]
        
        # Find column containing "Total" allocation
        total_col = next((col for col in df.columns if "total" in col), None)
        if total_col:
            df = df[["ministry", total_col]]
            df.rename(columns={"ministry": "Ministry", total_col: f"Total_{year}-{year+1}"}, inplace=True)
            
            # Match ministries and merge
            df = df.merge(common_ministries, left_on="Ministry", right_on="Ministry_05_06", how="inner")
            df.drop(columns=["Ministry_05_06"], inplace=True)
            df.rename(columns={"Matched_Ministry_20_21": "Ministry"}, inplace=True)
            
            if final_df.empty:
                final_df = df
            else:
                final_df = pd.merge(final_df, df, on="Ministry", how="outer")

# Save the final cleaned dataset
final_csv_path = os.path.join(budget_data_folder, "cleaned_budget_data.csv")
final_df.to_csv(final_csv_path, index=False)

print(f"Processed dataset saved at: {final_csv_path}")

Processed dataset saved at: /Users/vvmohith/Desktop/PROJECT/dataset/cleaned_budget_data.csv


In [12]:
import os
import pandas as pd

# Define dataset folder and output folder
dataset_folder = "./dataset/"
output_folder = "./filtered/"

# List of years to process
years = ["23-24", "22-23"]  # Add other years if needed

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Function to check if a row should be kept
def is_valid_row(value):
    return str(value).strip().startswith(tuple("0123456789")) or str(value).strip().lower() == "grand total"

# Loop through years
for year in years:
    input_path = os.path.join(dataset_folder, year, f"{year}.csv")
    output_path = os.path.join(output_folder, f"filtered_{year}.csv")

    # Check if file exists
    if not os.path.exists(input_path):
        print(f"❌ File not found: {input_path}, skipping...")
        continue

    print(f"📂 Processing file: {input_path}")

    # Load CSV
    df = pd.read_csv(input_path, delimiter=";", encoding="utf-8")

    # Keep only rows that start with a number OR are "Grand Total"
    df_filtered = df[df.iloc[:, 0].apply(is_valid_row)]

    # Save filtered data
    df_filtered.to_csv(output_path, index=False, sep=";")

    print(f"✅ Filtered data saved to {output_path}")


📂 Processing file: ./dataset/23-24/23-24.csv
✅ Filtered data saved to ./filtered/filtered_23-24.csv
📂 Processing file: ./dataset/22-23/22-23.csv
✅ Filtered data saved to ./filtered/filtered_22-23.csv


In [ ]:
# import pandas as pd
# import os

# # Path to datasets
# budget_data_folder = "/Users/vvmohith/Desktop/PROJECT/dataset/"  # Update with actual path
# match_file = "/Users/vvmohith/Desktop/PROJECT/matched_ministries.csv"  # Matched ministries file

# # Load matched ministries
# df_match = pd.read_csv(match_file)
# common_ministries = df_match[["Ministry_05_06", "Matched_Ministry_20_21"]]

# # Initialize final dataframe
# final_df = pd.DataFrame()

# # Get available years from folder names
# year_folders = [f for f in os.listdir(budget_data_folder) if os.path.isdir(os.path.join(budget_data_folder, f))]
# years_available = [int(folder.split("-")[0]) + 2000 for folder in year_folders if folder[:2].isdigit()]

# # Process each available year
# for year in sorted(years_available):
#     folder_name = f"{str(year)[-2:]}-{str(year+1)[-2:]}"
#     file_path = os.path.join(budget_data_folder, folder_name, f"{folder_name}.csv")

#     if os.path.exists(file_path):
#         print(f"\n🔹 Processing file: {file_path}")

#         # Try different delimiters
#         try:
#             df = pd.read_csv(file_path, sep=",", engine="python", skiprows=5)  # Skip first 5 rows if needed
#         except pd.errors.ParserError:
#             try:
#                 df = pd.read_csv(file_path, sep=";", engine="python", skiprows=5)
#             except pd.errors.ParserError:
#                 print(f"⚠️ Skipping file due to parsing error: {file_path}")
#                 continue

#         # Drop empty columns
#         df.dropna(axis=1, how="all", inplace=True)
        
#         # Detect column names dynamically
#         df.columns = [str(col).lower().strip() for col in df.columns]
#         print(f"📌 Available Columns: {df.columns.tolist()}")

#         # Find "Total" and "Ministry" columns
#         ministry_col = next((col for col in df.columns if "ministry" in col or "department" in col), None)
#         total_col = next((col for col in df.columns if "total" in col), None)

#         if not ministry_col or not total_col:
#             print(f"⚠️ No valid 'Total' or 'Ministry' column found in {file_path}. Skipping...")
#             continue

#         # Keep only Ministry and relevant "Total" column
#         df = df[[ministry_col, total_col]]
#         df.rename(columns={ministry_col: "Ministry", total_col: f"Total_{year}-{year+1}"}, inplace=True)

#         # Match ministries and merge
#         df = df.merge(common_ministries, left_on="Ministry", right_on="Ministry_05_06", how="inner")
#         df.drop(columns=["Ministry_05_06"], inplace=True)
#         df.rename(columns={"Matched_Ministry_20_21": "Ministry"}, inplace=True)

#         if final_df.empty:
#             final_df = df
#         else:
#             final_df = pd.merge(final_df, df, on="Ministry", how="outer")

# # Save the final cleaned dataset
# final_csv_path = os.path.join(budget_data_folder, "cleaned_budget_data.csv")
# final_df.to_csv(final_csv_path, index=False)

# print(f"\n✅ Processed dataset saved at: {final_csv_path}")
# print(f"📊 Final DataFrame shape: {final_df.shape}")



🔹 Processing file: /Users/vvmohith/Desktop/PROJECT/dataset/05-06/05-06.csv
📌 Available Columns: ['agricultural research and education', '748.98', '687.71', '1436.69', '900.00', '775.00', '1675.00', '1150.00', '792.00', '1942.00', 'unnamed: 10']
⚠️ No valid 'Total' or 'Ministry' column found in /Users/vvmohith/Desktop/PROJECT/dataset/05-06/05-06.csv. Skipping...

🔹 Processing file: /Users/vvmohith/Desktop/PROJECT/dataset/06-07/06-07.csv
📌 Available Columns: ['agricultural research and education', '816.01', '773.48', '1589.49', '1070.00', '830.00', '1900.00', '1350.00', '810.00', '2160.00']
⚠️ No valid 'Total' or 'Ministry' column found in /Users/vvmohith/Desktop/PROJECT/dataset/06-07/06-07.csv. Skipping...

🔹 Processing file: /Users/vvmohith/Desktop/PROJECT/dataset/07-08/07-08.csv
📌 Available Columns: ['agricultural research and education', '1046.75', '829.65', '1876.40', '1430.00', '846.00', '2276.00', '1620.00', '840.00', '2460.00', 'unnamed: 10']
⚠️ No valid 'Total' or 'Ministry' co

In [1]:
import pandas as pd
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def load_and_clean_csv(file_path, year):
    """Load CSV and return cleaned dataframe with year suffix in total column"""
    try:
        df = pd.read_csv(file_path, delimiter=',')
        # Remove numbering from ministry names if present
        df.iloc[:, 0] = df.iloc[:, 0].str.replace(r'^\d+\.\s+', '', regex=True)
        # Remove leading/trailing whitespace
        df.iloc[:, 0] = df.iloc[:, 0].str.strip()
        # Rename columns
        df.columns = ['Ministry', f'Total_{year}']
        return df
    except:
        try:
            df = pd.read_csv(file_path, delimiter=';')
            df.iloc[:, 0] = df.iloc[:, 0].str.replace(r'^\d+\.\s+', '', regex=True)
            df.iloc[:, 0] = df.iloc[:, 0].str.strip()
            df.columns = ['Ministry', f'Total_{year}']
            return df
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return None

def find_best_match(ministry_name, ministry_list, threshold=80):
    """Find best matching ministry name from list using fuzzy matching"""
    if pd.isna(ministry_name) or ministry_name.strip() == '':
        return None
    
    matches = process.extract(ministry_name, ministry_list, scorer=fuzz.token_sort_ratio)
    best_match = matches[0] if matches else None
    
    if best_match and best_match[1] >= threshold:
        return best_match[0]
    return None

# Path to dataset folder
dataset_folder = "/Users/vvmohith/Desktop/PROJECT/dataset-final/"

# Get list of all year folders
years = []
for year in range(5, 24):  # 05-06 to 23-24
    folder_name = f"{year:02d}-{(year+1):02d}"
    if os.path.exists(os.path.join(dataset_folder, folder_name)):
        years.append(folder_name)

# Initialize dictionary to store dataframes
dfs = {}

# Load all CSV files
for year in years:
    file_path = os.path.join(dataset_folder, year, f"{year}.csv")
    if os.path.exists(file_path):
        df = load_and_clean_csv(file_path, year)
        if df is not None:
            dfs[year] = df
            print(f"Loaded {year} with {len(df)} ministries")

# Use 21-22 as reference for ministry names
reference_year = "21-22"
reference_ministries = dfs[reference_year]['Ministry'].tolist()

# Create merged dataframe
merged_df = pd.DataFrame()
merged_df['Ministry'] = dfs[reference_year]['Ministry']

# Match and merge data from all years
for year in years:
    if year in dfs:
        df = dfs[year]
        matches = {}
        
        # Find matches for each ministry in current year
        for idx, row in df.iterrows():
            match = find_best_match(row['Ministry'], reference_ministries)
            if match:
                matches[row['Ministry']] = match
        
        # Create mapping series
        mapping_series = pd.Series(matches)
        
        # Map and merge
        df['Matched_Ministry'] = df['Ministry'].map(mapping_series)
        df = df.dropna(subset=['Matched_Ministry'])
        
        # Merge with final dataframe
        merged_df = merged_df.merge(
            df[['Matched_Ministry', f'Total_{year}']],
            left_on='Ministry',
            right_on='Matched_Ministry',
            how='left'
        )
        merged_df = merged_df.drop('Matched_Ministry', axis=1)

# Fill NaN values with 0
merged_df = merged_df.fillna(0)

# Save merged dataframe
output_path = os.path.join(dataset_folder, 'merged_budget_data.csv')
merged_df.to_csv(output_path, index=False)

print(f"\nMerged data saved to: {output_path}")
print(f"Final shape: {merged_df.shape}")
print("\nSample of merged data:")
print(merged_df.head())

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Loaded 05-06 with 94 ministries
Loaded 06-07 with 96 ministries
Loaded 07-08 with 123 ministries
Loaded 09-10 with 101 ministries
Loaded 10-11 with 101 ministries
Loaded 12-13 with 101 ministries
Loaded 13-14 with 102 ministries
Loaded 15-16 with 131 ministries
Loaded 16-17 with 127 ministries
Loaded 19-20 with 102 ministries
Loaded 20-21 with 104 ministries
Loaded 21-22 with 102 ministries
Loaded 22-23 with 96 ministries
Loaded 23-24 with 92 ministries

Merged data saved to: /Users/vvmohith/Desktop/PROJECT/dataset-final/merged_budget_data.csv
Final shape: (914, 15)

Sample of merged data:
                                            Ministry  Total_05-06  \
0  Department of Agriculture Cooperation and Farm...         0.00   
1  Department of Agriculture Cooperation and Farm...         0.00   
2  Department of Agricultural Research and Education      1942.00   
3                                      Atomic Energy      4995.86   
4  Ministry of Ayurveda Yoga and Naturopathy Unan...      

In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np

def standardize_ministry_names(df):
    # Create a mapping dictionary for similar ministry names
    ministry_mapping = {}
    ministries = df['Ministry'].unique()
    
    # First pass - group similar ministries
    for i in range(len(ministries)):
        if ministries[i] not in ministry_mapping:
            ministry_mapping[ministries[i]] = ministries[i]
            for j in range(i + 1, len(ministries)):
                # Use token_set_ratio to handle rearranged words
                similarity = fuzz.token_set_ratio(ministries[i], ministries[j])
                if similarity >= 80:  # Adjust threshold as needed
                    ministry_mapping[ministries[j]] = ministries[i]
    
    # Apply mapping
    df['Ministry'] = df['Ministry'].map(lambda x: ministry_mapping.get(x, x))
    
    # Aggregate duplicate ministries
    return df.groupby('Ministry', as_index=False).agg({
        col: 'max' if col.startswith('Total_') else 'first' 
        for col in df.columns if col != 'Ministry'
    })

# Read the CSV file
df = pd.read_csv("/Users/vvmohith/Desktop/PROJECT/dataset-final/merged_budget_data.csv")

# Clean and standardize ministry names
cleaned_df = standardize_ministry_names(df)

# Sort by ministry name
cleaned_df = cleaned_df.sort_values('Ministry')

# Save cleaned dataset
cleaned_df.to_csv("/Users/vvmohith/Desktop/PROJECT/dataset-final/cleaned_merged_budget_data.csv", index=False)

# Print some statistics
print(f"Original number of rows: {len(df)}")
print(f"Cleaned number of rows: {len(cleaned_df)}")
print("\nSample of merged ministries:")
print(cleaned_df.head())

Original number of rows: 914
Cleaned number of rows: 78

Sample of merged ministries:
                       Ministry  Total_05-06  Total_06-07  Total_07-08  \
0   Andaman and Nicobar Islands      1675.59      1977.93      1854.83   
1                 Atomic Energy      4995.86      5505.08      6130.00   
2                       Cabinet         0.00         0.00         0.00   
3  Central Vigilance Commission         0.00         0.00         0.00   
4                    Chandigarh       994.43      1032.90      1104.91   

   Total_09-10  Total_10-11  Total_12-13  Total_13-14 Total_15-16 Total_16-17  \
0      2689.23      2064.31      2982.65      3192.70     3748.38     4072.02   
1      7773.00      8521.00      9232.00      9833.32    10912.00    11682.48   
2         0.00         0.00         0.00         0.00           0           0   
3         0.00         0.00         0.00         0.00           0           0   
4      1772.56      1920.89      2546.96      3074.32     3457.1

In [3]:
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np

def clean_ministry_name(name):
    """Clean ministry name by removing prefixes and standardizing format"""
    name = str(name).strip()
    # Remove numbering at start
    name = name.split('.', 1)[-1].strip()
    # Remove common prefixes
    prefixes = ['Department of', 'Ministry of']
    for prefix in prefixes:
        if name.startswith(prefix):
            name = name[len(prefix):].strip()
    return name

def are_ministries_similar(name1, name2, threshold=85):
    """Check if two ministry names are similar using fuzzy matching"""
    if pd.isna(name1) or pd.isna(name2):
        return False
    return fuzz.token_sort_ratio(str(name1).lower(), str(name2).lower()) >= threshold

def combine_budget_data(files_dict):
    """Process and combine budget data from multiple years"""
    all_data = []
    
    # Process each year's data
    for year, file_path in files_dict.items():
        try:
            # Try different delimiters
            for delimiter in [',', ';']:
                try:
                    df = pd.read_csv(file_path, delimiter=delimiter)
                    break
                except:
                    continue
            
            # Clean column names
            df.columns = [col.lower().strip() for col in df.columns]
            ministry_col = next(col for col in df.columns if 'ministry' in col or 'department' in col)
            total_col = next(col for col in df.columns if 'total' in col)
            
            # Select and rename columns
            year_df = df[[ministry_col, total_col]].copy()
            year_df.columns = ['Ministry', f'Total_{year}']
            
            # Clean ministry names
            year_df['Ministry'] = year_df['Ministry'].apply(clean_ministry_name)
            
            all_data.append(year_df)
            print(f"Processed {year} data: {len(year_df)} entries")
            
        except Exception as e:
            print(f"Error processing {year} data: {str(e)}")
    
    # Merge all years
    final_df = all_data[0]
    for df in all_data[1:]:
        final_df = pd.merge(final_df, df, on='Ministry', how='outer')
    
    return final_df

# Define the years and file paths
years = [f"{str(year)[-2:]}-{str(year+1)[-2:]}" for year in range(2005, 2024)]
files_dict = {
    year: f"/Users/vvmohith/Desktop/PROJECT/dataset-final/{year}/{year}.csv"
    for year in years
}

# Process the data
merged_df = combine_budget_data(files_dict)

# Fill NaN values with 0
merged_df = merged_df.fillna(0)

# Sort by ministry name
merged_df = merged_df.sort_values('Ministry')

# Add numbering
merged_df.insert(0, 'No.', range(1, len(merged_df) + 1))

# Save the final cleaned dataset
output_path = "/Users/vvmohith/Desktop/PROJECT/dataset-final/final_budget_data.csv"
merged_df.to_csv(output_path, index=False)

print(f"\nFinal dataset saved to: {output_path}")
print(f"Total number of unique ministries: {len(merged_df)}")
print("\nFirst few entries:")
print(merged_df.head().to_string())

Processed 05-06 data: 94 entries
Processed 06-07 data: 96 entries
Processed 07-08 data: 123 entries
Processed 08-09 data: 123 entries
Processed 09-10 data: 101 entries
Processed 10-11 data: 101 entries
Processed 11-12 data: 101 entries
Processed 12-13 data: 101 entries
Processed 13-14 data: 102 entries
Processed 14-15 data: 102 entries
Processed 15-16 data: 131 entries
Processed 16-17 data: 127 entries
Processed 17-18 data: 127 entries
Processed 18-19 data: 127 entries
Processed 19-20 data: 102 entries
Processed 20-21 data: 104 entries
Processed 21-22 data: 102 entries
Processed 22-23 data: 96 entries
Processed 23-24 data: 92 entries

Final dataset saved to: /Users/vvmohith/Desktop/PROJECT/dataset-final/final_budget_data.csv
Total number of unique ministries: 119964

First few entries:
   No.                         Ministry  Total_05-06  Total_06-07 Total_07-08 Total_08-09  Total_09-10  Total_10-11  Total_11-12  Total_12-13  Total_13-14  Total_14-15 Total_15-16 Total_16-17 Total_17-18

In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np
from collections import defaultdict

def clean_ministry_name(name):
    """Clean ministry name by removing prefixes and standardizing format"""
    if pd.isna(name):
        return ""
    name = str(name).strip()
    # Remove numbering and special characters
    name = ' '.join(word for word in name.split() if not word.startswith(('(', '#', '@')))
    name = name.split('.', 1)[-1].strip()
    
    # Remove common prefixes
    prefixes = ['Department of', 'Ministry of']
    for prefix in prefixes:
        if name.lower().startswith(prefix.lower()):
            name = name[len(prefix):].strip()
    return name

def group_similar_ministries(all_files):
    """First step: Group similar ministry names across all years"""
    all_ministries = set()
    ministry_groups = {}
    
    # First collect all unique ministry names
    for file_path in all_files:
        try:
            df = pd.read_csv(file_path)
            ministry_col = df.columns[0]  # First column is usually ministry
            ministries = df[ministry_col].apply(clean_ministry_name)
            all_ministries.update(ministries.dropna().unique())
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    # Remove empty strings
    all_ministries = {m for m in all_ministries if m.strip()}
    
    # Group similar ministries
    processed = set()
    for ministry in all_ministries:
        if ministry in processed:
            continue
            
        similar_ministries = {ministry}
        for other in all_ministries:
            if other != ministry and other not in processed:
                ratio = fuzz.token_sort_ratio(ministry.lower(), other.lower())
                if ratio >= 85:  # Adjust threshold as needed
                    similar_ministries.add(other)
                    processed.add(other)
        
        if len(similar_ministries) > 1:
            # Use the most recent (usually more detailed) name as canonical
            canonical = max(similar_ministries, key=len)
            ministry_groups[canonical] = similar_ministries
            processed.update(similar_ministries)
            
    return ministry_groups

def combine_budget_data(ministry_groups, all_files):
    """Second step: Combine budget data using ministry groups"""
    # Initialize DataFrame with canonical ministry names
    canonical_names = list(ministry_groups.keys())
    final_df = pd.DataFrame({'Ministry': canonical_names})
    
    # Process each year's file
    for file_path in sorted(all_files):
        try:
            year = file_path.split('/')[-2]  # Extract year from path
            df = pd.read_csv(file_path)
            ministry_col = df.columns[0]
            total_col = df.columns[1]
            
            # Create year's data with canonical names
            year_data = defaultdict(float)
            
            for _, row in df.iterrows():
                ministry = clean_ministry_name(row[ministry_col])
                if not ministry:
                    continue
                    
                # Find canonical name for this ministry
                canonical = None
                for can, group in ministry_groups.items():
                    if ministry in group or any(fuzz.token_sort_ratio(ministry, m) >= 85 for m in group):
                        canonical = can
                        break
                
                if canonical:
                    try:
                        value = float(str(row[total_col]).replace(',', ''))
                        year_data[canonical] += value
                    except (ValueError, TypeError):
                        continue
            
            # Add year's data to final DataFrame
            final_df[f'Total_{year}'] = final_df['Ministry'].map(year_data)
            print(f"Processed {year}")
            
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    return final_df

# Get all CSV files
import glob
all_files = sorted(glob.glob("/Users/vvmohith/Desktop/PROJECT/dataset-final/*/[0-9][0-9]-[0-9][0-9].csv"))

print("Step 1: Grouping similar ministries...")
ministry_groups = group_similar_ministries(all_files)

print("\nStep 2: Combining budget data...")
final_df = combine_budget_data(ministry_groups, all_files)

# Clean up and format
final_df = final_df.fillna(0)
final_df = final_df.sort_values('Ministry')
final_df.insert(0, 'No.', range(1, len(final_df) + 1))

# Save results
output_path = "/Users/vvmohith/Desktop/PROJECT/dataset-final/final_cleaned_budget.csv"
final_df.to_csv(output_path, index=False)

print(f"\nFinal dataset saved to: {output_path}")
print(f"Total number of unique ministries: {len(final_df)}")
print("\nSimilar ministry groups found:")
for canonical, group in ministry_groups.items():
    if len(group) > 1:
        print(f"\n{canonical}:")
        print("  - " + "\n  - ".join(sorted(group - {canonical})))

Step 1: Grouping similar ministries...

Step 2: Combining budget data...
Processed 05-06
Processed 06-07
Processed 07-08
Processed 09-10
Processed 10-11
Processed 12-13
Processed 13-14
Processed 15-16
Processed 16-17
Processed 19-20
Processed 20-21
Processed 21-22
Processed 22-23
Processed 23-24

Final dataset saved to: /Users/vvmohith/Desktop/PROJECT/dataset-final/final_cleaned_budget.csv
Total number of unique ministries: 32

Similar ministry groups found:

Agriculture and Cooperation:
  - Agriculture & Cooperation

Dadra and Nagar Haveli:
  - Dadra & Nagar Haveli

UNION TERRITORIES + B):
  - UNION TERRITORIES

Pharmaceuticals@:
  - Pharmaceuticals

Personnel, Public Grievances and Pensions:
  - Personnel Public Grievances and Pensions

Agriculture Cooperation and Farmers' Welfare:
  - Agriculture Cooperation and Farmers Welfare

Heavy Industries:
  - Heavy Industry

Economic Affairs provisions):
  - Economic Affairs Provisions)

Total Central Assistance for States and UTs + III):
  

In [ ]:
import pandas as pd
import os
import glob
from fuzzywuzzy import fuzz
import numpy as np
from collections import defaultdict

def clean_ministry_name(name):
    """Clean ministry name by removing numbering and standardizing format"""
    if pd.isna(name):
        return ""
    
    name = str(name).strip()
    
    # Remove numbering at start (like "1." or "23.")
    if '.' in name and name.split('.')[0].strip().isdigit():
        name = name.split('.', 1)[1].strip()
    
    # Strip special characters and extra spaces
    name = ' '.join(name.split())
    return name

def identify_ministry_groups():
    """First step: Identify groups of similar ministries"""
    all_files = []
    for year in range(5, 24):  # 05-06 to 22-23
        year_folder = f"{year:02d}-{(year+1):02d}"
        csv_path = f"/Users/vvmohith/Desktop/PROJECT/dataset-final/{year_folder}/{year_folder}.csv"
        if os.path.exists(csv_path):
            all_files.append(csv_path)
    
    # Collect all unique ministry names
    all_ministries = set()
    for file_path in all_files:
        try:
            df = pd.read_csv(file_path)
            ministry_col = df.columns[0]  # First column usually contains ministry names
            cleaned_names = df[ministry_col].apply(clean_ministry_name)
            all_ministries.update(cleaned_names.dropna().unique())
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    # Remove empty entries
    all_ministries = {m for m in all_ministries if m and not m.isspace()}
    
    # Build standardization mapping using fuzzy matching
    ministry_mapping = {}
    canonical_ministries = {}
    
    # Sort ministries by length (descending) to prefer more detailed names
    sorted_ministries = sorted(all_ministries, key=len, reverse=True)
    
    # Identify groups of similar ministries
    for ministry in sorted_ministries:
        # Skip if already mapped to a canonical name
        if ministry in ministry_mapping:
            continue
        
        # Start a new group with this ministry
        similarity_group = [ministry]
        for other in sorted_ministries:
            if other != ministry and other not in ministry_mapping:
                # Check if similar
                similarity = fuzz.token_sort_ratio(ministry.lower(), other.lower())
                if similarity >= 85:
                    similarity_group.append(other)
        
        # Use most descriptive name as canonical name
        canonical = similarity_group[0]
        
        # Special case for prefixes
        for name in similarity_group:
            if name.startswith("Department of") or name.startswith("Ministry of"):
                canonical = name
                break
        
        # Register the group
        canonical_ministries[canonical] = similarity_group
        for name in similarity_group:
            ministry_mapping[name] = canonical
    
    print(f"Found {len(canonical_ministries)} canonical ministry names")
    return ministry_mapping

def process_budget_files(ministry_mapping):
    """Process all budget files and create consolidated dataset"""
    # Initialize DataFrame with columns for each year
    columns = ['Ministry']
    for year in range(5, 24):
        columns.append(f'Total_{year:02d}-{(year+1):02d}')
    
    result_df = pd.DataFrame(columns=columns)
    
    # Get unique canonical ministry names
    canonical_names = set(ministry_mapping.values())
    result_df['Ministry'] = sorted(canonical_names)
    result_df = result_df.set_index('Ministry')
    
    # Process each year's budget file
    for year in range(5, 24):
        year_str = f"{year:02d}-{(year+1):02d}"
        file_path = f"/Users/vvmohith/Desktop/PROJECT/dataset-final/{year_str}/{year_str}.csv"
        
        if os.path.exists(file_path):
            print(f"Processing {year_str} data...")
            try:
                df = pd.read_csv(file_path)
                
                # Identify ministry and budget columns
                ministry_col = df.columns[0]
                budget_col = df.columns[1] if len(df.columns) > 1 else None
                
                if budget_col:
                    # Create a dictionary to store budget values by canonical ministry name
                    year_data = defaultdict(float)
                    
                    # Process each row
                    for _, row in df.iterrows():
                        ministry_name = clean_ministry_name(row[ministry_col])
                        if ministry_name in ministry_mapping:
                            canonical = ministry_mapping[ministry_name]
                            try:
                                # Convert budget to float, handling commas and other formatting
                                budget_val = str(row[budget_col]).replace(',', '')
                                budget_val = float(''.join(c for c in budget_val if c.isdigit() or c == '.'))
                                year_data[canonical] += budget_val
                            except (ValueError, TypeError):
                                pass
                    
                    # Add data to results DataFrame
                    year_col = f'Total_{year_str}'
                    for ministry, value in year_data.items():
                        result_df.loc[ministry, year_col] = value
                
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
    
    # Fill NaN values with 0
    result_df = result_df.fillna(0)
    
    # Reset index to make Ministry a column
    result_df = result_df.reset_index()
    
    # Add numbering
    result_df.insert(0, 'No.', range(1, len(result_df) + 1))
    
    return result_df

# Main execution flow
print("Step 1: Identifying similar ministry names...")
ministry_mapping = identify_ministry_groups()

print("\nStep 2: Processing budget files...")
final_df = process_budget_files(ministry_mapping)

# Save the result
output_path = "/Users/vvmohith/Desktop/PROJECT/dataset-final/normalized_budget_data.csv"

print(f"\nNormalized budget data saved to: {output_path}")
print(f"Total unique ministries: {len(final_df)}")

# Display statistics
print("\nTop 10 ministries by total budget (all years):")
final_df['Total'] = final_df.iloc[:, 2:].sum(axis=1)
top_ministries = final_df.sort_values('Total', ascending=False).head(10)
for _, row in top_ministries.iterrows():
    print(f"{row['No.']}. {row['Ministry']}: {row['Total']:,.2f}")

print("\nSample normalized ministry groups:")
# Show a few examples of ministry name normalization
for canonical, names in list({name: [k for k, v in ministry_mapping.items() if v == name] 
                             for name in set(ministry_mapping.values())}.items())[:5]:
    if len(names) > 1:
        print(f"\n{canonical}:")
        for name in sorted(names):
            if name != canonical:
                print(f"  - {name}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Step 1: Identifying similar ministry names...
Found 235 canonical ministry names

Step 2: Processing budget files...
Processing 05-06 data...
Processing 06-07 data...
Processing 07-08 data...
Processing 09-10 data...
Processing 10-11 data...
Processing 12-13 data...
Processing 13-14 data...
Processing 15-16 data...
Processing 16-17 data...
Processing 19-20 data...
Processing 20-21 data...
Processing 21-22 data...
Processing 22-23 data...
Processing 23-24 data...

Normalized budget data saved to: /Users/vvmohith/Desktop/PROJECT/dataset-final/normalized_budget_data.csv
Total unique ministries: 235

Top 10 ministries by total budget (all years):
97. Grand Total: 25,870,098.15
101. I. CENTRAL SECTOR: 6,185,235.88
111. Interest Payments: 4,203,587.42
209. b. Economic Affairs (Centralised Provisions): 3,273,487.90
26. CENTRAL SECTOR - TOTAL: 1,944,115.76
44. Defence Services: 1,423,309.55
198. Transfers to States: 1,319,613.73
45. Defence Services (Revenue): 1,122,953.00
202. V. GRAND TOTAL 

/var/folders/ty/wzy1hvwd7bd4plh9mwvg9yyc0000gn/T/ipykernel_1121/2048136811.py:139: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result_df = result_df.fillna(0)
